In [ ]:
# 플라스크 통신을 위한 라이브러리
from flask import Flask, request, jsonify, redirect
from flask_cors import CORS

# 얼굴 인식을 위한 라이브러리
from deepface import DeepFace

# 얼굴 모자이크 처리를 위한 라이브러리
import cv2
import numpy as np
import base64

# 플라스크 통신 시작
app = Flask( __name__ )
CORS( app )

@app.route( "/mosaic", methods = [ "GET", "POST" ] ) 
# ================= 시작 ===============================
def test() : 
    if request.method == "GET": # GET 방식으로 통신 시
        data = dict(request.args)
    else : # post 방식으로 통신 시
        data = dict(request.form)

    # 불러올 데이터(이미지)
    image_path = f'C:/Users/smhrd1/Desktop/Web_jsp/.metadata/.plugins/org.eclipse.wst.server.core/tmp0/wtpwebapps/goming123/images/{data["img"]}'

    
    # DeepFace를 사용하여 얼굴 영역 추출
    detected_faces = DeepFace.extract_faces(image_path, detector_backend='retinaface', enforce_detection=False)
    
    # 모자이크를 위해 OpenCV 이미지 로드
    image = cv2.imread(image_path)
    
    # 감지된 각 얼굴에 대해 모자이크 처리
    for face in detected_faces:
        x = face['facial_area'].get('x')
        y = face['facial_area'].get('y')
        w = face['facial_area'].get('w')
        h = face['facial_area'].get('h') # 감지된 얼굴 영역의 좌표 값 추출

        if x !=0 and y !=0:
            # 좌표 값에 맞게 모자이크 진행
            face_roi = image[y:y+h, x:x+w]
            blur_face = cv2.GaussianBlur(face_roi, (99, 99), 30)
            image[y:y+h, x:x+w] = blur_face

    # 모자이크 된 image 파일을 mosaic_파일명 형태로 저장
    cv2.imwrite(f'C:/Users/smhrd1/Desktop/Web_jsp/.metadata/.plugins/org.eclipse.wst.server.core/tmp0/wtpwebapps/goming123/post/mosaic_{data["img"]}', image)

    # 통신 종료 후 다시 업로드 서비스로 redirect
    return redirect(f"http://localhost:8085/goming/UpdateService?idx={data['idx']}&img=mosaic_{data['img']}")


# 서버 실행
if __name__ == "__main__":
    app.run(
        host = "192.168.219.62", 
    port = 5050 
    )


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.219.62:5050
Press CTRL+C to quit
192.168.219.62 - - [28/Jun/2024 16:50:44] "GET / HTTP/1.1" 404 -
192.168.219.62 - - [28/Jun/2024 16:50:44] "GET /favicon.ico HTTP/1.1" 404 -
192.168.219.62 - - [28/Jun/2024 16:50:48] "GET /test HTTP/1.1" 404 -
[2024-06-28 16:50:52,619] ERROR in app: Exception on /mosaic [GET]
Traceback (most recent call last):
  File "C:\Users\smhrd1\anaconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\smhrd1\anaconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\smhrd1\anaconda3\Lib\site-packages\flask_cors\extension.py", line 178, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "C:\Users\smhrd1